# Quickstart Tutorial: Neurodesktop & BIDS Conversion

Welcome! This tutorial will guide you through:
- Launching Neurodesktop tools
- Converting XNAT DICOM data to BIDS format
- Running common neuroimaging pipelines (MRIQC, fMRIPrep)

## Table of Contents
- [1. Intro](#1.-Intro)
- [2. Data Conversion](#2.-Data-Conversion)
  - [2.1 Install xnat_dcm2bids](#2.1-Install-xnat_dcm2bids)
  - [2.2 Important Tools & Links](#2.2-Important-Tools-&-Links)
  - [2.3 Configure XNAT Connection](#2.3-Configure-XNAT-Connection)
  - [2.4 Download Session List](#2.4-Download-Session-List)
  - [2.5 Prepare BIDS Directory](#2.5-Prepare-BIDS-Directory)
  - [2.6 Setup Config File](#2.6-Setup-Config-File)
  - [2.7 Convert Sessions](#2.7-Convert-Sessions)
- [3. Software & Pipelines](#3.-Software-&-Pipelines)
  - [3.1 MRIQC](#3.1-MRIQC)
  - [3.2 fMRIPrep](#3.2-fMRIPrep)
  - [3.3 Preinstalled Software](#3.3-Preinstalled-software)
  - [3.4 Loading Software from Neurodesk Repo](#3.4-Loading-software-from-Neurodesk-Repo)
  - [3.5 Feel Free to Install New Software](#3.5-Feel-free-to-install-new-software)
- [FAQ](#FAQ)
  

## 1. Intro

This tutorial will guide you through launching Neurodesktop tools using Jupyter Notebook, however you can also run those tools in Terminal. Just use the commands below without exclamation mark "❗"


Start by launching Neurodesktop and choosing one of the following:
- **Terminal**
- **Jupyter Notebook**
- **Remote Desktop (Neurodesktop GUI)**

In the Neurodesktop GUI you can run MATLAB, FSL, Freeview, and other programs directly. See [Software](#3.-Software-&-Pipelines) for details.

## 2. Data Conversion

### 2.1 Install `xnat_dcm2bids`

You can install this package on Your own MacOS, Windows, or Linux using pip:


In [ ]:
!pip install https://github.com/nencki-lobi/xnat_dcm2bids/archive/refs/heads/master.zip

### 2.2 Important Tools & Links
   - **xnat_dcm2bids** - downloads data from a selected XNAT session (e.g., `f490f566-2124-46`) and converts it to BIDS format using `dcm2bids`:  
`xnat_dcm2bids --bids-dir ~/bids-dir --config ./code/config.json f490f566-2124-46 03 01` 
   - **dcm2bids_scaffold** - prepares a new BIDS directory  
     `dcm2bids_scaffold -o /home/jovyan/bids-dir`
   - **xnat-get** - downloads all DICOM files from XNAT  
     `xnat-get -p mj_ris -t ./sourcedata`
   - **xnat_getcsv** - downloads session list to CSV  
     `xnat_getcsv -o mj_ris.csv mj_ris`
   - **lobi_script** - run scripts from [lobi-mri-scripts](https://github.com/nencki-lobi/lobi-mri-scripts)  
     `lobi_script ls` will display list of available scripts eg.   
     `lobi_script run_mriqc.sh 03 ~/bids-dir ~/bids-dir/derivatives/mriqc`

Find out more in [xnat_dcm2bids](https://github.com/nencki-lobi/xnat_dcm2bids) repository.

### 2.3 Configure XNAT Connection

xnat_dcm2bids tools use specific authentication method. It is neccesarry to initialize the connection for the first time **in Terminal - type `xnat-ls`** and the provide the url `xnat.nencki.edu.pl` and Your credentials.
Expected output is a list of all available projects.

### 2.4 Download Session List

To download list of all available xnat sessions from project "public" type:

In [11]:
%cd ~
!xnat_getcsv -o list.csv public

/home/jovyan
🟢 Data saved to list.csv


In [12]:
#display content of "list.csv"
import pandas as pd
df = pd.read_csv("list.csv", dtype=str, index_col=0)
df.head(10)

,Subject_ID,subject,session
MR ID,,,
7c2afbc5-763e-4a,LOBI_fingertapping,NaN,NaN
tstdtst,test_lobi,NaN,NaN


### 2.5 Prepare BIDS Directory

In [2]:
bidsdir="/home/jovyan/bids-dir"

In [4]:
!dcm2bids_scaffold -o {bidsdir}

INFO    | --- dcm2bids_scaffold start ---
INFO    | Running the following command: /home/jovyan/.local/bin/dcm2bids_scaffold -o /home/jovyan/bids-dir
INFO    | OS version: Linux-5.15.0-153-generic-x86_64-with-glibc2.39
INFO    | Python version: 3.12.11 | packaged by conda-forge | (main, Jun  4 2025, 14:45:31) [GCC 13.3.0]
INFO    | dcm2bids version: 3.2.0
INFO    | Checking for software update
INFO    | Currently using the latest version of dcm2bids.
INFO    | The files used to create your BIDS directory were taken from https://github.com/bids-standard/bids-starter-kit. 

INFO    | Tree representation of /home/jovyan/bids-dir/
INFO    | /home/jovyan/bids-dir/
INFO    | ├── code/
INFO    | ├── derivatives/
INFO    | ├── sourcedata/
INFO    | ├── tmp_dcm2bids/
INFO    | │   └── log/
INFO    | │       └── scaffold_20251013-162658.log
INFO    | ├── .bidsignore
INFO    | ├── CHANGES
INFO    | ├── dataset_description.json
INFO    | ├── participants.json
INFO    | ├── participants.tsv
INFO   

In [ ]:
%cd {bidsdir}

/home/jovyan/bids-dir


In [ ]:
!ls -1 {bidsdir}

CHANGES
README
code
dataset_description.json
derivatives
participants.json
participants.tsv
sourcedata
sub-03
tmp_dcm2bids


### 2.6 Setup Config File

Usually we provide You with the customized configuration for dcm2bids. Pre-cooked configurations are available in the [lobi-mri-scripts repository](https://github.com/nencki-lobi/lobi-mri-scripts/tree/main/dcm2bids). If You have already cloned the repository (run lobi_script), then You can link one of the configuration files as follows:

``ln -sf ~/lobi-mri-scripts/dcm2bids/mjris.json {bidsdir}/code/config.json``

Below is the example of a configuration for a basic experiment consisting of T1w, task fmri and fieldmaps. It's rather straighforward.

In [5]:
%%writefile {bidsdir}/code/config.json
{
  "descriptions": [
    {
      "datatype": "anat",
      "suffix": "T1w",
      "criteria": {
        "SeriesDescription": "anat_t1w"
      }
    },
    {
      "id": "fmri",
      "datatype": "func",
      "suffix": "bold",
      "criteria": {
        "SeriesDescription": "task-*"
      }
    },
    {
      "datatype": "fmap",
      "suffix": "epi",
      "criteria": {
        "SeriesDescription": "SpinEchoFieldMap*"
      },
      "sidecar_changes": {
        "intendedFor": ["fmri"]
      }
    }
  ]
}


Writing /home/jovyan/bids-dir/code/config.json


### 2.7 Convert Sessions

**Single session example:**
- XNAT session ID: `tstdtst`
- Subject: `03`
- Session: `01`


In [ ]:
!xnat_dcm2bids --output-dir {bidsdir} --config ./code/config.json tstdtst 03 01

**Multiple sessions from CSV:**


In [14]:
#"echo" will print the command without running it. Remove "echo" to execute.
!while IFS=',' read -r id _ sub ses _; do \
  echo xnat_dcm2bids --config ./code/config.json "$id" "$sub" 1; \
done < ~/list.csv

xnat_dcm2bids --config ./code/config.json MR ID subject 1
xnat_dcm2bids --config ./code/config.json 7c2afbc5-763e-4a  1
xnat_dcm2bids --config ./code/config.json tstdtst  1


## 3. Software & Pipelines

In [15]:
!lobi_script ls

Scripts in /home/jovyan/lobi-mri-scripts:
freesurfer_init.sh
run_dcm2bids_noauto.sh
run_fmriprep_kids_alicja.sh
run_fmriprep_min.sh
run_mriqc.sh


### 3.1 MRIQC

Run MRIQC for a single subject:


In [ ]:
!lobi_script run_mriqc.sh 03 {bidsdir} {bidsdir}/derivatives/mriqc

### 3.2 fMRIPrep

Run fMRIPrep for multiple subjects from CSV:


In [ ]:
!tail -n +2 ~/mj_ris.csv | parallel --colsep ',' echo lobi_script run_fmriprep_min.sh {{3}} {bidsdir} {bidsdir}/derivatives/fmriprep

### 3.3 Preinstalled software

Graphical tools do not work efficiently via `module load` from Neurodesktop repository. Instead, follow these steps to make it work from local:
- **MATLAB**: run in Terminal with `./matlab`
- **FSL**: configure first with `source ~/shared_storage/fsl.sh`, then run eg. `fsleyes`

### 3.4 Loading software from Neurodesk Repo

Other programs including graphical viewers such as Freeview, mrtrix etc. are not available locally and should be run in Terminal from Neurodesktop via `module load XXX` or shortly `ml XXX` eg. `ml ants`. In Jupyter the syntax is as follows:

In [2]:
import module
await module.load('ants')

In [ ]:
!antsRegistration

### 3.5 Feel free to install new software

Use `sudo apt update; sudo apt install XXX` to install any program You want. You don't need to ask :-)  
The software will became unavailable after shutdown of the environment.

# FAQ

1. **Where are my files?**

> All files are stored on the DRAGON storage system in Mikołajki. No matter whether you work in `$HOME`, `./shared_storage`, or use your dedicated disk — your files will be secure and available at full speed in each case.

2. **How can I upload my own files?**

> * Use drag & drop for multiple files
> * Use `xnat_dcm2bids`, etc., to download DICOMs from XNAT
> * Use a dedicated storage disk, which can be accessed both from Neurodesk and from your personal computer.

3. **Where should I store my files?**

> * The `$HOME` directory (1 TB) is dedicated to your personal needs. No one except you and the administrator can access or delete files stored there.
> * The `./shared_storage` directory (5 TB) is dedicated to collaboration and sharing data between users. Anyone can access or delete files stored there.
> * Dedicated directories may be created upon request. See below for how to request a dedicated drive or home folder.

4. **I want a dedicated disk for my research group project. How do I request it?**

> You need to submit a request via the Helpdesk. Please provide:
>
> * **Location → DRAGON**
> * Lab ID (e.g., N601, N417)
> * Name of the PI and all members
> * Project name
> * Required size (in TB)
>
> Your disk will be accessible from Neurodesk and any other computer on the local network.

5. **How can I access my dedicated disk?**

> You can access the files using standard protocols from Neurodesk as well as from other computers in the Nencki network. To mount your dedicated drive in Neurodesk, follow these steps:
> 1) In `$HOME`, create a file named `cifs_mounts.conf`.
> 2) In the file, provide the relevant information:
>```
>//dragon.nencki.edu.pl/417-maestro/PROJECT,PROJECT,USERNAME@nencki.ibd, PASSWORD
>```
> **Do not use spaces** or other irrelevant characters!  The second PROJECT is the target directory in Your workspace.
> 4) Apply the changes: **File → Hub Control → Stop and Start**
> 5) You should now see the new directory `PROJECT` in `$HOME`.

6. **What happens if I shut down or restart the environment?**

> Don’t worry — your data will be preserved. However, all installed programs will be removed because a fresh version of Neurodesktop will be loaded.

7. **How can I access HTML report from mriqc and fmriprep?**

> You can start http server in the directory consisting Your html files as usual using python ``python -m http.server 8000``. You will be able to access the server via Jupyter's proxy at [neurodesk.nencki.edu.pl/user/YOUR_NAME/proxy/8000/](https://neurodesk.nencki.edu.pl/user/YOUR_NAME/proxy/8000/). Mind the ending **slash** in the URL.

8. **Where I can find more information about the new tools?**

> Check out the Neurodesktop website and tutorials: [https://neurodesk.org/example-notebooks/intro.html](https://neurodesk.org/example-notebooks/intro.html)  
> Visit our GitHub repositories:
> * [https://github.com/nencki-lobi/lobi-mri-scripts](https://github.com/nencki-lobi/lobi-mri-scripts)
> * [https://github.com/nencki-lobi/xnat_dcm2bids](https://github.com/nencki-lobi/xnat_dcm2bids)


